In [1]:
from __future__ import division, print_function, absolute_import, unicode_literals

In [2]:
# packages initialization

import torch
from torch.utils.data import TensorDataset, DataLoader
from torch import optim
import torch.nn as nn
import torch.nn.functional as F

from scipy import sparse
import numpy as np
# time
# math
# heapq

## Configuration

In [3]:
class Config(object):
    def __init__(self):
        self.data_dir = './data/CAMRa2011/'
        self.embedding_size = 32
#         self.epoch = 30
        self.epoch = 1
        self.num_negatives = 6
        self.batch_size = 256
        self.lr = 0.000005
        self.drop_ratio = 0.2
        self.topK = 5

In [4]:
config = Config()

## DataLoader

In [5]:
class CAMRa2011Dataset(object):
    """CAMRa2011 dataset"""
    
    def __init__(self, dataset_dir):
        
        self.pathes = {
            'train': {
                'user': dataset_dir + "userRatingTrain.txt",
                'group': dataset_dir + "groupRatingTrain.txt"
            },
            'test': {
                'user': dataset_dir + "userRatingTest.txt",
                'user_negative': dataset_dir + "userRatingNegative.txt",
                'group': dataset_dir + "groupRatingTest.txt",
                'group_negative': dataset_dir + "groupRatingNegative.txt",
            },
            'group_user': dataset_dir + "groupMember.txt"
        }
        
        # get the mapping of users and groups
        # format: {gid: [uid, uid, ..], gid: [uid, uid, ..], ...}
        self.group_members = self.get_group_user_mapping()
        
        # get interaction matrix from uid-iid training set
        # train_user_matrix[uid, iid] = [1 | 0]
        self.train_user_matrix = self.get_interaction_matrix(self.pathes['train']['user'])
        
        # format: [[uid, iid], [uid, iid], ...]
        # only pairs of users & items have interactions would appear in the list
        self.test_user_list = self.get_interaction_list(self.pathes['test']['user'])
        
        # format: [[uid, ...], [uid, ...], ...]
        # test_user_negative_list & test_user_list follow the same order
        # e.g. test_user_negative_list[0] is for test_user_list[0]
        self.test_user_negative_list = self.get_negatives(self.pathes['test']['user_negative'])
        
        # get interaction matrix from gid-iid training set
        self.train_group_matrix = self.get_interaction_matrix(self.pathes['train']['group'])

        # pairs of group & item to be tested
        self.test_group_list = self.get_interaction_list(self.pathes['test']['group'])

        self.test_group_negative_list = self.get_negatives(self.pathes['test']['group_negative'])
        
    def get_user_dataloader(self, batch_size=256, shuffle=True):
        
        users, positives_negatives = self.get_train_instances(self.train_user_matrix)
        dataset = TensorDataset(
            torch.tensor(users, dtype=torch.float),
            torch.tensor(positives_negatives, dtype=torch.float))
        
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
        
        return loader
    
    def get_group_dataloader(self, batch_size=256, shuffle=True):
        
        groups, positives_negatives = self.get_train_instances(self.train_group_matrix)
        
        dataset = TensorDataset(
            torch.tensor(groups, dtype=torch.float),
            torch.tensor(positives_negatives, dtype=torch.float))
        
        loader = DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)
        
        return loader
    
    # get number of groups, users and items
    def get_sizes(self):
        group_size = len(self.group_members)
        (user_size, item_size) = self.train_user_matrix.shape
        
        return (group_size, user_size, item_size)
        
    def get_train_instances(self, interaction_matrix, num_negatives=6):
        
        users, positive_items, negative_items = [], [], []
        
        num_users, num_items = interaction_matrix.shape
        
        for (uid, iid) in interaction_matrix.keys():
            
            # positive instance
            for _ in range(num_negatives):
                
                # positive instances
                positive_items.append(iid)
                
                # negative instances ---> need to be fixed
                negative_iid = np.random.randint(num_items+1)
                while (uid, negative_iid) in interaction_matrix:
                    negative_iid = np.random.randint(num_items+1) # re-generate an negative iid
                negative_items.append(negative_iid)
                
                # users
                users.append(uid)

        positives_negatives = [[positive_iid, negative_iid] for positive_iid, negative_iid in zip(positive_items, negative_items)]
        
        return users, positives_negatives
        
        
    def get_group_user_mapping(self):
    
        mapping = {}

        # read mapping file
        with open(self.pathes['group_user'], 'r') as file:

            line = file.readline().strip()
            while line != None and line != "":

                # sample line format: [gid] [uid 1],[uid 2],[uid 3],[uid 4]
                sequences = line.split(' ')
                gid = int(sequences[0])
                mapping[gid] = []
                for uid in sequences[1].split(','):
                    mapping[gid].append(int(uid))
                line = file.readline().strip()

        return mapping

    # parse all interactions in dataset to 2D sparse matrix
    def get_interaction_matrix(self, rating_file_path):

        # get number of users and items
        num_users, num_items = 0, 0
        with open(rating_file_path, "r") as file:

            line = file.readline()
            while line != None and line != "":
                arr = line.split(" ")
                uid, iid = int(arr[0]), int(arr[1])
                num_users = max(num_users, uid)
                num_items = max(num_items, iid)
                line = file.readline()

        # construct interaction matrix
        # dok_matrix: Dictionary Of Keys based sparse matrix, an efficient structure for constructing sparse matrices incrementally.
        matrix = sparse.dok_matrix((num_users + 1, num_items + 1), dtype=np.float32) # iid and uid starts from 1
        with open(rating_file_path, "r") as file:
            line = file.readline()
            while line != None and line != "":
                arr = line.split(" ")
                if len(arr) > 2:
                    uid, iid, rating = int(arr[0]), int(arr[1]), int(arr[2])
                    if (rating > 0):
                        matrix[uid, iid] = 1.0
                else:
                    uid, iid = int(arr[0]), int(arr[1])
                    matrix[uid, iid] = 1.0
                line = file.readline()

        return matrix
    
    # parse all interactions in dataset to list
    def get_interaction_list(self, rating_file_path):

        interaction_list = []
        with open(rating_file_path, "r") as file:
            line = file.readline()
            while line != None and line != "":
                arr = line.split(" ")
                uid, iid = int(arr[0]), int(arr[1])
                interaction_list.append([uid, iid])
                line = file.readline()

        return interaction_list

    # parse negative sample lists for pairs in test set
    # negative samples: the items which never been interacted
    # the order of returned sample lists must be paired with test list
    def get_negatives(self, file_path):

        negative_samples_list = []

        with open(file_path, "r") as file:

            line = file.readline()
            while line != None and line != "":
                arr = line.split(" ")

                negative_iids = []
                for iid in arr[1:]:
                    negative_iids.append(int(iid))

                negative_samples_list.append(negative_iids)
                line = file.readline()

        return negative_samples_list

In [73]:
## test

a = [(1, 23), (12, 34)]

na = torch.tensor(a)


True

In [6]:
dataset = CAMRa2011Dataset(config.data_dir)

In [7]:
num_group, num_user, num_item = dataset.get_sizes()

## AGREE Model (Attentive Group Representation)

In [8]:
# embedding networks

class UserEmbeddingLayer(nn.Module):
    def __init__(self, num_users, embedding_dim):
        super(UserEmbeddingLayer, self).__init__()
        self.userEmbedding = nn.Embedding(num_users, embedding_dim)

    def forward(self, uids):
        user_embedded = self.userEmbedding(uids)
        return user_embedded
    
class ItemEmbeddingLayer(nn.Module):
    def __init__(self, num_items, embedding_dim):
        super(ItemEmbeddingLayer, self).__init__()
        self.itemEmbedding = nn.Embedding(num_items, embedding_dim)

    def forward(self, iids):
        item_embedded = self.itemEmbedding(iids)
        return item_embedded
    
class GroupEmbeddingLayer(nn.Module):
    def __init__(self, num_groups, embedding_dim):
        super(GroupEmbeddingLayer, self).__init__()
        self.groupEmbedding = nn.Embedding(num_groups, embedding_dim)

    def forward(self, gids):
        group_embedded = self.groupEmbedding(gids)
        return group_embedded

In [9]:
# attention network

class AttentionLayer(nn.Module):
    def __init__(self, embedding_dim, drop_ratio=0):
        super(AttentionLayer, self).__init__()
        
        self.linear1 = nn.Linear(embedding_dim, 16)
        self.linear2 = nn.Linear(16, 1)
        self.dropout = nn.Dropout(p=drop_ratio)
        
        self.linear = nn.Sequential(
            nn.Linear(embedding_dim, 16),
            nn.ReLU(),
            nn.Dropout(drop_ratio),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        x = self.linear2(x)
        weights = F.softmax(x.view(1, -1), dim=1)

        return weights

In [10]:
# final layers of AGREE for prediction

class PredictLayer(nn.Module):
    def __init__(self, embedding_dim, drop_ratio=0):
        super(PredictLayer, self).__init__()
        
        self.linear1 = nn.Linear(embedding_dim, 8)
        self.dropout = nn.Dropout(p=drop_ratio)
        self.linear2 = nn.Linear(8, 1)

    def forward(self, x):
        
        x = F.relu(self.linear1(x))
        x = self.dropout(x)
        out = self.linear2(x)
        
        return out

In [69]:
class AGREE(nn.Module):
    def __init__(self, num_users, num_items, num_groups, embedding_dim, group_member_mapping, drop_ratio):
        super(AGREE, self).__init__()
        
        self.user_embedding = UserEmbeddingLayer(num_users, embedding_dim)
        self.item_embedding = ItemEmbeddingLayer(num_items, embedding_dim)
        self.group_embedding = GroupEmbeddingLayer(num_groups, embedding_dim)
        self.attention = AttentionLayer(2 * embedding_dim, drop_ratio)
        self.predict = PredictLayer(3 * embedding_dim, drop_ratio)
        
        self.group_members = group_member_mapping
        
        self.num_users = num_users
        self.num_groups = len(self.group_members)
        
        # initial model's parameters
        for m in self.modules():
            
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=1) # normal distribution
            if isinstance(m, nn.Embedding):
                nn.init.xavier_normal_(m.weight) # Glorot initialization

    def forward(self, gids, uids, iids):
        
        # group prediction
        if (gids is not None) and (uids is None):
            output = self.group_forward(gids, iids)
        # user prediction
        else:
            output = self.user_forward(uids, iids)
            
        return output

    # group forwarding
    def group_forward(self, gids, iids):
        
        # group_embeds = Variable(torch.Tensor())
        group_embeddeds = torch.empty(0)
        
        # generate embedding vector for item
        item_embeddeds = self.item_embedding(
            iids.clone().type(torch.long).unsqueeze(dim=1)) # shape: (batch_size, 1, embedding_dim)
        
        # get attentive group embedding
        for gid, iid in zip(gids, iids):
            member_uids = self.group_members[gid.item()]
            
            # generate user embedding vector
            member_embeddeds = self.user_embedding(
                torch.tensor(member_uids, dtype=torch.long).unsqueeze(dim=1)) # shape: (batch_size, 1, embedding_dim)
            
            # generate item embedding vector
            items = [iid.item() for _ in member_uids]
            item_embeddeds = self.item_embedding(
                torch.tensor(items, dtype=torch.long).unsqueeze(dim=1)) # shape: (batch_size, 1, embedding_dim)
            
            # get attentive weights for each user-item pair
            user_item_embeddeds = torch.cat((member_embeddeds, item_embeddeds), dim=2) # shape: (num_member, 1, 2 * embedding_dim)
            user_item_embeddeds = user_item_embeddeds.squeeze(dim=1) # shape: (num_member, 2 * embedding_dim)
            attentive_weights = self.attention(user_item_embeddeds) # shape: (num_member, 1)
            
            # aggregation
            member_embeddeds = member_embeddeds.squeeze(dim=1) # shape: (batch_size, embedding_dim)
            aggregated_user_item_embeddeds = torch.matmul(attentive_weights.T, member_embeddeds) # shape: (1, embedding_dim)
            
            # generate group-item embedding vector
            group_embedded = self.group_embedding(torch.tensor([[gid.item()]], dtype=torch.long)) # shape: (1, embedding_dim)
            
            # group embedding = user embedding aggregation + group preference embedding
            aggregated_all_embedded = aggregated_user_item_embeddeds + group_embedded # shape: (1, embedding_dim)
            
            # append group embedding
            group_embeddeds = torch.cat((group_embeddeds, group_embedded), dim=0) # shape: (batch_size, embedding_dim)
            
        # element-wise product: group-item interaction
        interacted_embeddeds = torch.mul(group_embeddeds, item_embeddeds) # shape: (batch_size, embedding_dim)
        
        # pooling: [group x item, group, item]
        pooled_embeddeds = torch.cat((interacted_embeddeds, group_embeddeds, item_embeddeds), dim=1) # shape: (batch_size, 3*embedding_dim)
        
        y = torch.sigmoid(self.predict(pooled_embeddeds))
        return y

    # user forwarding
    def user_forward(self, uids, iids):
        
        # uids.shape: (batch_size, )
        # iids.shape: (batch_size, )
        
        print("iids dtype")
        
        # generate user embedding vectors
        user_embeddeds = self.user_embedding(
            uids.clone().type(torch.long).unsqueeze(dim=1)) # (batch_size, 1, embedding_dim)
        
        # generate item embedding vectors
        item_embeddeds = self.item_embedding(
            iids.clone().type(torch.long).unsqueeze(dim=1)) # (batch_size, 1, embedding_dim)
        
        # element-wise product: user-item interactions
        interacted_embeddeds = torch.mul(user_embeddeds, item_embeddeds) # (batch_size, 1, embedding_dim)
        
        # pooling: [user x item, group, item]
        pooled_embeddeds = torch.cat((interacted_embeddeds, user_embeddeds, item_embeddeds), dim=2) # (batch_size, 1, 3 * embedding_dim)
        
        # reshape x from (batch_size, 1, 3 * embedding_dim) to (batch_size, 3 * embedding_dim)
        pooled_embeddeds = pooled_embeddeds.squeeze(dim=1)
        
        y = torch.sigmoid(self.predict(pooled_embeddeds))
        
        return y

## Training

In [68]:
# model training procedure
def training(model, dataloader, epoch_id, config, input_type):
    
    # user trainning
    learning_rate = config.lr

    # lr decay: halve for every five epochs
    for _ in range(0, epoch_id, 5):
        learning_rate /= 2

    # create optimizer
    optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)
    
    print("Epoch {}, lr = {}, input_type = {}".format(epoch_id, learning_rate, input_type))

    losses = []
    for batch_id, (uids, piids_niids) in enumerate(dataloader):
        
        print(batch_id, end=" ")
        
        # Data Load
        p_iids = piids_niids[:, 0]
        n_iids = piids_niids[:, 1]
        
        # Forward
        if input_type == 'user':
            positive_predictions = model(None, uids, p_iids)
            negative_predictions = model(None, uids, n_iids)
        elif input_type == 'group':
            positive_predictions = model(uids, None, p_iids)
            negative_predictions = model(uids, None, n_iids)

        optimizer.zero_grad()
        
        # calculate loss
        loss = torch.mean((positive_predictions - negative_predictions -1) **2)
        
        # back propagation
        loss.backward()
        optimizer.step()
        
        # record loss history
        losses.append(loss.item())
    
    return np.mean(losses)

In [66]:
agree = AGREE(num_user, num_item, num_group, config.embedding_size, dataset.group_members, config.drop_ratio)
print("AGREE at embedding size: {}, epoch: {}, NDCG % HR: Top-{}".format(config.embedding_size, config.epoch, config.topK))

AGREE at embedding size: 32, epoch: 1, NDCG % HR: Top-5


In [67]:
history = []
for epoch in range(config.epoch):
    
    # set the model in train mode (some network like dropout will behave differently in train mode / evaluaiton mode)
    agree.train(mode=True)
    
    # train the model using user interactions
    user_loss = training(agree, dataset.get_user_dataloader(batch_size=config.batch_size, shuffle=True),
                 epoch, config, 'user')
    
    history.append(user_loss)
    
    # train the model using group & members interactions
    group_loss = training(agree, dataset.get_group_dataloader(batch_size=config.batch_size, shuffle=True),
                 epoch, config, 'group')
    
    history.append(group_loss)
    
    print("Losses: {}, {}".format(user_loss, group_loss))

Epoch 0, lr = 5e-06, input_type = user
0 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
1 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
2 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
3 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
4 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
5 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
6 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
7 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
8 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_embeddeds.shape: torch.Size([256, 1, 96])
9 pooled_embeddeds.shape: torch.Size([256, 1, 96])
pooled_emb

IndexError: index out of range in self